In [1]:
input_file = open("input.txt").read()

# input_file = """???.### 1,1,3
# .??..??...?##. 1,1,3
# ?#?#?#?#?#?#?#? 1,3,1,6
# ????.#...#... 4,1,1
# ????.######..#####. 1,6,5
# ?###???????? 3,2,1
# """

In [2]:
input_file = [
    [a, tuple(map(int, b.split(",")))]
    for a, b in [line.split(" ") for line in input_file.splitlines()]
]
print(input_file)

[['.??.?#??##???.', (1, 6)], ['?#???.#??#?????.?', (3, 1, 3, 1, 1)], ['??#????#??#???.??', (4, 7, 1)], ['??#?.?#?#???#?#??', (1, 11)], ['??????????', (1, 4, 1)], ['?#?#????..??????', (6, 4)], ['?#?#?.#?#???????..?', (5, 1, 1, 3, 2, 1)], ['?.#??#??.??????.?', (6, 1, 1, 1)], ['##???###?.##??', (2, 4, 2)], ['???.#??#????.???', (3, 7, 2)], ['????#??..?#', (1, 3, 1)], ['.?..#????????#?????', (1, 5, 6)], ['?#..??##??.???', (1, 5, 2)], ['?#???#???#', (1, 1, 2)], ['#???#??..?????.??.?.', (5, 1, 1, 1, 2, 1)], ['??#??.?#??????.', (2, 2, 1, 1, 1)], ['??????.???..??#??', (1, 2, 1, 4)], ['.?#??#???#??#?????.', (12, 3)], ['??#???#?.?##?.??', (6, 3)], ['?#?#?.??#????', (3, 1, 1, 1)], ['??#???.#??.#.', (3, 2, 1)], ['??#...??..', (3, 2)], ['#?#.??????#.???????', (3, 2, 1, 1, 7)], ['?#?#.?????#????', (3, 5)], ['?#??.???.?.?.????', (4, 1, 1, 3)], ['????????.##?#.????', (2, 1, 4, 2)], ['?#??#.??????????#?', (4, 3, 1, 3)], ['.#?#?.???..?', (3, 1, 1, 1)], ['?..##.???#????.', (2, 2, 2)], ['.#??#?#??#?.?', (9

In [3]:
input_file = [["?".join([a] * 5), b * 5] for a, b in input_file]

# Recursive approach


We trim down the input sequence as we find suitable

For example, and starting `...` shall we discarded.

Any starting `###.` will be matched with the int sequence to verify if they are a match. We pop them out, and recursively compute the rest

We apply memoization with Python's @functools.cache, where the computation for same sequence can be easily reused.

In [7]:
from functools import cache
import re


@cache
def get_sol(s_s: str, s_i: tuple[int, ...]) -> int:
    s_s = s_s.lstrip(".")

    match = re.match(r"(#+)(?=\.|$)(.*)", s_s)
    if match:
        prefix, remaining = match[1], match[2]
        if s_i and len(prefix) == s_i[0]:  # checked off first set of #s
            return get_sol(remaining, s_i[1:])
        else:
            return 0

    elif s_s == "":
        return int(len(s_i) == 0)

    elif (r_idx := s_s.index("?")) >= 0:
        try:
            return get_sol(f"{s_s[:r_idx]}.{s_s[r_idx+1:]}", s_i) + get_sol(
                f"{s_s[:r_idx]}#{s_s[r_idx+1:]}", s_i
            )
        except Exception as ex:
            print(s_s, r_idx)
            raise ex

    else:
        raise RuntimeError

In [8]:
from tqdm.notebook import tqdm

spring_vals = [get_sol(*row) for row in tqdm(input_file)]
sum(spring_vals)

  0%|          | 0/1000 [00:00<?, ?it/s]

1909291258644

Some older attempts at solving the problem. To be fair this is less optimized and probably can't leverage memoization for computing n_solutions.

As this solution do not trim down the spring map and int values on compute. Every combination will be very unique across all inputs.

In [9]:
# def verify(s_s: str, s_i: tuple[int, ...]) -> bool:
#     return [s.count("#") for s in s_s.split(".") if s] == s_i

# def get_solutions(s_s: str, s_i: SpringSeq) -> int:
#     n_rhash = sum(s_i) - s_s.count("#")
#     if n_rhash == 0:
#         s_s = s_s.replace("?", ".")

#     n_wc = s_s.count("?")
#     if n_wc == 0:
#         return int(verify(s_s, s_i))


#     r_idx = s_s.index("?")
#     s_s1 = f"{s_s[:r_idx]}#{s_s[r_idx+1:]}"
#     s_s2 = f"{s_s[:r_idx]}.{s_s[r_idx+1:]}"

#     return get_solutions(s_s1, s_i) + get_solutions(s_s2, s_i)